In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Using the requests library

In [2]:
url = 'https://rldaggie.github.io/sample-html/'
res = requests.get(url)

### Status Codes

In [3]:
res.status_code

200

### Creating a `BeautifulSoup` object

In [4]:
soup = BeautifulSoup(res.content, 'lxml')

# `soup.find()`

Returns either:

1. A soup object of the first match
2. `None`

In [5]:
h1 = soup.find('h1')

In [6]:
type(h1)

bs4.element.Tag

In [7]:
h1.text

'This is an h1'

In [8]:
h1.attrs

{'class': ['foobar'], 'id': 'title'}

In [9]:
if h1:
    print(h1.text)

This is an h1


# `soup.find_all()`

Returns a **_LIST_** of soup objects that match your query

In [10]:
h1_tags = soup.find_all('h1')

In [11]:
[tag.text for tag in h1_tags]

['This is an h1', 'This is yet another heading.']

In [12]:
[tag.attrs for tag in h1_tags]

[{'class': ['foobar'], 'id': 'title'}, {'class': ['foobar']}]

# Creating a `pandas` DataFrame from a scrape

In [13]:
#people = [
#    {'name': 'Bethany', 'market': 'BOS'},
#    {'name': 'Tucker', 'market': 'NYC'}
#]

people = {'name': ['Bethany','Tucker'], 
     'market': ['BOS','NYC']}

pd.DataFrame(people)

,name,market
0,Bethany,BOS
1,Tucker,NYC


### Todo List

In [14]:
ol = soup.find('ol', {'class': 'done'})
ol

<ol class="done">
<li>Mow lawn</li>
<li class="foobar"><span>Take out compost</span></li>
<li><span>Create scraping lecture</span></li>
</ol>

In [15]:
todos = []
for li in ol.find_all('li'):
    todo = {}
    todo['task'] = li.text
    todos.append(todo)
pd.DataFrame(todos)

,task
0,Mow lawn
1,Take out compost
2,Create scraping lecture


### GA Directory

In [16]:
table = soup.find('table', {'id': 'directory'})

In [17]:
people = []
for row in table.find('tbody').find_all('tr'):
    person = {}
    person['name'] = row.find('a').text.strip()
    person['email'] = row.find('a').attrs['href'].replace('mailto:', '')
    person['role'] = row.find('td').text.strip()
    
    people.append(person)
pd.DataFrame(people)

,name,email,role
0,Praveen,praveen@ga.co,Student
1,Fred,fred@ga.co,Student
2,Homer,homer@ga.co,Student
3,Kyle,kyle@ga.co,Student
4,Sam,sam@ga.co,Student
5,Javier,javier@ga.co,Student
6,Nengkuan,nengkuan@ga.co,Student
7,Kieth,kieth@ga.co,Student
8,Bola,bola@ga.co,Student
9,Steve,steve@ga.co,Student


### Basketball Reference

In [18]:
url = 'https://www.basketball-reference.com/'
res = requests.get(url)
print(res.status_code)

soup = BeautifulSoup(res.content, 'lxml')

200


In [19]:
teams = []
for conf in ['E', 'W']:
    table = soup.find('table', {'id': 'confs_standings_'+conf})
    for row in table.find('tbody').find_all('tr'):
        team = {}
        team['slug'] = row.find('a').text
        team['name'] = row.find('a').attrs['title']
        team['wins'] = row.find_all('td')[2].text
        team['wins'] = row.find('td', {'data-stat': 'wins'}).text
        team['losses'] = row.find('td', {'data-stat': 'losses'}).text
        team['rank'] = row.find('span').text.strip()[1:-1]
        team['conference'] = conf

        teams.append(team)
df = pd.DataFrame(teams)
df

,slug,name,wins,losses,rank,conference
0,MIL,Milwaukee Bucks,53,12,1,E
1,TOR,Toronto Raptors,46,18,2,E
2,BOS,Boston Celtics,43,21,3,E
3,MIA,Miami Heat,41,24,4,E
4,IND,Indiana Pacers,39,26,5,E
5,PHI,Philadelphia 76ers,39,26,6,E
6,BRK,Brooklyn Nets,30,34,7,E
7,ORL,Orlando Magic,30,35,8,E
8,WAS,Washington Wizards,24,40,9,E
9,CHO,Charlotte Hornets,23,42,10,E


In [20]:
df.dtypes

slug          object
name          object
wins          object
losses        object
rank          object
conference    object
dtype: object